In [1]:
pip install numpy pandas matplotlib seaborn tensorflow keras pydot graphviz

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import keras
from tensorflow.keras.layers import Input, Dense, Lambda, Layer, Activation
from tensorflow.keras.layers import BatchNormalization
from keras.models import Model
from keras import backend as K
from keras import metrics, optimizers
from keras.callbacks import Callback

import pydot
import graphviz
from keras.utils import plot_model
from keras_tqdm import TQDMNotebookCallback
from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot

/Users/tyronemariano/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
#print keras and tensorflow versions for compatability

print(keras.__version__)
print(tf.__version__)

3.5.0
2.17.0


In [5]:
#sets the plotting backend for matplotlib to be inline
#inline: meaning it will be displayed directly into the notebook instead of other window

%matplotlib inline

In [9]:
#uses certain style sheet which is 'seaborn-v0_8-notebook'

plt.style.use('seaborn-v0_8-notebook')

In [10]:
#seaborn styling

sns.set(style="white", color_codes=True)
sns.set_context("paper", rc={"font.size":14,"axes.titlesize":15,"axes.labelsize":20,
                             'xtick.labelsize':14, 'ytick.labelsize':14})